In [2]:
import pandas as pd
from prophet import Prophet
from prophet.plot import plot_plotly, plot_components_plotly
import json
from prophet.serialize import model_to_json, model_from_json

import yfinance as yf
import datetime
from datetime import date, timedelta
today = date.today()

d1 = today.strftime("%Y-%m-%d")
end_date = d1
d2 = date.today() - timedelta(days=730)
d2 = d2.strftime("%Y-%m-%d")
start_date = d2

DataSet = yf.download('BTC-USD', 
                      start=start_date, 
                      end=end_date, 
                      progress=False)
DataSet["Date"] = DataSet.index
DataSet = DataSet[["Date", "Open", "High", "Low", "Close", "Adj Close", "Volume"]]
DataSet.reset_index(drop=True, inplace=True)
print(DataSet.head())


        Date         Open         High          Low        Close    Adj Close  \
0 2020-05-11  8755.535156  9033.470703  8374.323242  8601.795898  8601.795898   
1 2020-05-12  8610.385742  8949.898438  8569.643555  8804.477539  8804.477539   
2 2020-05-13  8805.387695  9317.878906  8805.387695  9269.987305  9269.987305   
3 2020-05-14  9271.329102  9793.268555  9255.035156  9733.721680  9733.721680   
4 2020-05-15  9734.291016  9755.828125  9261.398438  9328.197266  9328.197266   

        Volume  
0  57119858802  
1  42142717533  
2  45558144023  
3  56426907637  
4  48158802327  


In [3]:
DataSet.rename(columns={'Date': 'ds'}, inplace=True)
DataSet.rename(columns={'Close': 'y'}, inplace=True)
DataSet.head()

,ds,Open,High,Low,y,Adj Close,Volume
0,2020-05-11,8755.535156,9033.470703,8374.323242,8601.795898,8601.795898,57119858802
1,2020-05-12,8610.385742,8949.898438,8569.643555,8804.477539,8804.477539,42142717533
2,2020-05-13,8805.387695,9317.878906,8805.387695,9269.987305,9269.987305,45558144023
3,2020-05-14,9271.329102,9793.268555,9255.035156,9733.721680,9733.721680,56426907637
4,2020-05-15,9734.291016,9755.828125,9261.398438,9328.197266,9328.197266,48158802327


In [4]:
df = DataSet[['ds', 'y']]
df.head()

,ds,y
0,2020-05-11,8601.795898
1,2020-05-12,8804.477539
2,2020-05-13,9269.987305
3,2020-05-14,9733.721680
4,2020-05-15,9328.197266


In [5]:
m = Prophet()
m.fit(df)

INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


In [6]:
future = m.make_future_dataframe(periods=365)
future.tail()

,ds
1091,2023-05-07
1092,2023-05-08
1093,2023-05-09
1094,2023-05-10
1095,2023-05-11


In [7]:
forecast = m.predict(future)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

,ds,yhat,yhat_lower,yhat_upper
1091,2023-05-07,-45121.707101,-110742.798013,22122.747145
1092,2023-05-08,-46131.265060,-111405.529397,21379.038089
1093,2023-05-09,-47171.367477,-113636.935058,20322.584546
1094,2023-05-10,-48053.387546,-112968.244284,20140.130920
1095,2023-05-11,-49339.255407,-116513.428203,19678.537677


In [8]:
plot_plotly(m, forecast)

In [9]:
plot_components_plotly(m, forecast)

In [10]:
with open('models\Bitcoin.json', 'w') as fout:
    json.dump(model_to_json(m), fout)


In [11]:
m = model_from_json(json.loads(open('models/Bitcoin.json').read()))

In [12]:
m.predict(future)

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,weekly,weekly_lower,weekly_upper,yearly,yearly_lower,yearly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
0,2020-05-11,12431.372271,14921.969083,21460.017746,12431.372271,12431.372271,5704.056096,5704.056096,5704.056096,-21.278885,-21.278885,-21.278885,5725.334981,5725.334981,5725.334981,0.0,0.0,0.0,18135.428366
1,2020-05-12,12470.970654,13916.977649,20626.877699,12470.970654,12470.970654,4760.107975,4760.107975,4760.107975,-47.889562,-47.889562,-47.889562,4807.997537,4807.997537,4807.997537,0.0,0.0,0.0,17231.078628
2,2020-05-13,12510.569036,13323.659050,19792.691199,12510.569036,12510.569036,4007.025018,4007.025018,4007.025018,126.073844,126.073844,126.073844,3880.951173,3880.951173,3880.951173,0.0,0.0,0.0,16517.594054
3,2020-05-14,12550.167419,12161.646246,18937.594140,12550.167419,12550.167419,2884.630037,2884.630037,2884.630037,-69.219252,-69.219252,-69.219252,2953.849289,2953.849289,2953.849289,0.0,0.0,0.0,15434.797456
4,2020-05-15,12589.765802,11220.664704,18124.489185,12589.765802,12589.765802,2003.163799,2003.163799,2003.163799,-33.198916,-33.198916,-33.198916,2036.362714,2036.362714,2036.362714,0.0,0.0,0.0,14592.929601
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1091,2023-05-07,-54820.470104,-110768.692741,25448.240011,-119861.470909,15067.694821,9698.763003,9698.763003,9698.763003,24.307849,24.307849,24.307849,9674.455154,9674.455154,9674.455154,0.0,0.0,0.0,-45121.707101
1092,2023-05-08,-55033.180058,-111837.495302,23822.446284,-120389.484727,15037.232554,8901.914997,8901.914997,8901.914997,-21.278885,-21.278885,-21.278885,8923.193883,8923.193883,8923.193883,0.0,0.0,0.0,-46131.265060
1093,2023-05-09,-55245.890011,-112509.467241,23676.710865,-120938.463681,15013.882894,8074.522534,8074.522534,8074.522534,-47.889562,-47.889562,-47.889562,8122.412096,8122.412096,8122.412096,0.0,0.0,0.0,-47171.367477
1094,2023-05-10,-55458.599964,-114008.807975,22578.151850,-121487.758440,15340.675115,7405.212418,7405.212418,7405.212418,126.073844,126.073844,126.073844,7279.138574,7279.138574,7279.138574,0.0,0.0,0.0,-48053.387546
